**HackStory Hackathon 2023**

**HATE SPEECH RECOGNITION**

In [10]:
#necessary Python libraries
from nltk.util import pr
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import re
import nltk
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
stopword=set(stopwords.words('english'))
data = pd.read_csv("hate_speech.csv")
print(data.head())

   Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0      3            0                   0        3      2   
1           1      3            0                   3        0      1   
2           2      3            0                   3        0      1   
3           3      3            0                   2        1      1   
4           4      6            0                   6        0      1   

                                               tweet  
0  !!! RT @mayasolovely: As a woman you shouldn't...  
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...  
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...  
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...  
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...  


**Create Labels**
[0-->Hate speech , 1-->Offensive Language , 2-->No Hate and Offensive]

In [11]:
data["labels"] = data["class"].map({0: 0, 
                                    1: 1, 
                                    2: 2})
print(data.head())

   Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0      3            0                   0        3      2   
1           1      3            0                   3        0      1   
2           2      3            0                   3        0      1   
3           3      3            0                   2        1      1   
4           4      6            0                   6        0      1   

                                               tweet  labels  
0  !!! RT @mayasolovely: As a woman you shouldn't...       2  
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...       1  
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...       1  
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...       1  
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...       1  


-->Selecting Tweets and Labels

In [12]:
data = data[["tweet", "labels"]]
print(data.head())

                                               tweet  labels
0  !!! RT @mayasolovely: As a woman you shouldn't...       2
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...       1
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...       1
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...       1
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...       1


-->Refining text [removing puctuation, seperating words]

In [13]:
def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
data["tweet"] = data["tweet"].apply(clean)

**MODEL TRAINING** 
[split the dataset into training and test sets]

In [14]:
x = np.array(data["tweet"])
y = np.array(data["labels"])

cv = CountVectorizer()
X = cv.fit_transform(x) # Fit/Insert the Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)

DecisionTreeClassifier()

**TEST MODEL**

In [15]:
sample = input("Enter the Tweet: ")
data = cv.transform([sample]).toarray()
print(clf.predict(data))

Enter the Tweet: 
[2]


In [17]:
import tkinter as tk
from tkinter import filedialog

file_path = None

def open_file():
    global file_path
    file_path = filedialog.askopenfilename(filetypes=[("Text Files", "*.txt")])
    if file_path:
        with open(file_path, 'r') as file:
            text = file.read()
            text_box.delete('1.0', tk.END)  # Clear existing text
            text_box.insert(tk.END, text)

def save_file():
    if file_path:
        with open(file_path, 'w') as file:
            file.write(text_box.get('1.0', tk.END))

def check_text():
    selected_text = ""
    if file_path:
        with open(file_path, 'r') as file:
            selected_text = file.read()
    else:
        selected_text = text_box.get('1.0', tk.END)
        
    data = cv.transform([selected_text]).toarray()
    data = data.reshape(1, -1)  # Reshape the data to 2D
    prediction = clf.predict(data)
    prediction_text.delete('1.0', tk.END)
    prediction_text.insert(tk.END, str(prediction))

# Create the main window
window = tk.Tk()
window.title("Text Reader")

# Create a text box to display and edit the text
text_box = tk.Text(window, height=10, width=50)
text_box.pack()

# Create an "Open" button
open_button = tk.Button(window, text="Open", command=open_file)
open_button.pack()

# Create a "Save" button
save_button = tk.Button(window, text="Save", command=save_file)
save_button.pack()

# Create a "Check" button
check_button = tk.Button(window, text="Check", command=check_text)
check_button.pack()

# Create a text box to display the prediction result
prediction_text = tk.Text(window, height=1, width=50)
prediction_text.pack()

# Start the GUI event loop
window.mainloop()
